In [1]:
import numpy as np
import time
from igraph import *

In [2]:
def get_graph_object(G):
    #This function converts the graph we saved into the graph object of igraph library
    (V,E) = G
    g = Graph(directed=True)
    g.add_vertices(range(len(V)))

    source = []
    target = []
    for v in V:
        degree = len(E[v])
        source = source + [v]*degree
        target = target + E[v]
    
    g.add_edges(zip(source,target))
    return g

In [3]:
def IC(Gt,S,p_sum,N_t,t,mc):
    
    g = get_graph_object(Gt)
    
    # Loop over the Monte-Carlo Simulations
    spread = []
    pulled_arms = {}
    for i in range(mc):
        
        # Simulate propagation process      
        new_active, A = S[:], S[:]
        while new_active:

            # For each newly active node, find its neighbors that become activated
            new_ones = []
            for node in new_active:
                
                # Determine neighbors that become infected
                np.random.seed(i)
                neighbours = g.neighbors(node,mode="out")
                p_neighbours = []
                for neigh in neighbours:
                    p_neighbours.append(p_sum[node][neigh]/N_t[node][neigh] + np.sqrt(3*np.log10(t)/2*N_t[node][neigh]))
                success = np.random.uniform(0,1,len(neighbours)) < p_neighbours
                pulled_arms[node] = list(set(np.extract(success, g.neighbors(node,mode="out"))) - set(new_ones))
                new_ones += list(np.extract(success, g.neighbors(node,mode="out")))
                
            new_active = list(set(new_ones) - set(A))
            
            # Add newly activated nodes to the set of activated nodes
            A += new_active
            
        spread.append(len(A))
        
    return(np.mean(spread)),pulled_arms

In [4]:
def greedy(Gt,k,p_sum,N_t,t,mc):

    g = get_graph_object(Gt)

    S, spread, timelapse, start_time = [], [], [], time.time()
    
    # Find k nodes with largest marginal gain
    for _ in range(k):

        # Loop over nodes that are not yet in seed set to find biggest marginal gain
        best_spread = 0
        for j in set(range(g.vcount()))-set(S):

            # Get the spread
            s,_ = IC(Gt,S + [j],p_sum,N_t,t,mc)

            # Update the winning node and spread so far
            if s > best_spread:
                best_spread, node = s, j

        # Add the selected node to the seed set
        S.append(node)
        
        # Add estimated spread and elapsed time
        spread.append(best_spread)
        timelapse.append(time.time() - start_time)

    return(S,spread,timelapse)

In [ ]:
def IC2(Gt,S,p_in,mc):
    
    g = get_graph_object(Gt)
    
    # Loop over the Monte-Carlo Simulations
    spread = []
    pulled_arms = {}
    for i in range(mc):
        
        # Simulate propagation process      
        new_active, A = S[:], S[:]
        while new_active:

            # For each newly active node, find its neighbors that become activated
            new_ones = []
            for node in new_active:
                
                # Determine neighbors that become infected
                np.random.seed(i)
                neighbours = g.neighbors(node,mode="out")
                p_neighbours = []
                for neigh in neighbours:
                    p_neighbours.append(p_in[node][neigh])
                success = np.random.uniform(0,1,len(neighbours)) < p_neighbours
                pulled_arms[node] = list(set(np.extract(success, g.neighbors(node,mode="out"))) - set(new_ones))
                new_ones += list(np.extract(success, g.neighbors(node,mode="out")))
                
            new_active = list(set(new_ones) - set(A))
            
            # Add newly activated nodes to the set of activated nodes
            A += new_active
            
        spread.append(len(A))
        
    return(np.mean(spread)),pulled_arms

In [ ]:
def greedy2(Gt,k,p_in,mc):

    g = get_graph_object(Gt)

    S, spread, timelapse, start_time = [], [], [], time.time()
    
    # Find k nodes with largest marginal gain
    for _ in range(k):

        # Loop over nodes that are not yet in seed set to find biggest marginal gain
        best_spread = 0
        for j in set(range(g.vcount()))-set(S):

            # Get the spread
            s,_ = IC2(Gt,S + [j],p_in,mc)

            # Update the winning node and spread so far
            if s > best_spread:
                best_spread, node = s, j

        # Add the selected node to the seed set
        S.append(node)
        
        # Add estimated spread and elapsed time
        spread.append(best_spread)
        timelapse.append(time.time() - start_time)

    return(S,spread,timelapse)